# Loss Functions
> Custom LightningCV loss functions

In [ ]:
# default_exp core.losses

<IPython.core.display.Javascript object>

In [ ]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [ ]:
# hide
import warnings
from nbdev.showdoc import *
from nbdev.export import *

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [ ]:
# export
from typing import *

import torch
from torch import nn
from torch import Tensor
import torch.nn.functional as F
from fvcore.nn import sigmoid_focal_loss
from fastcore.all import store_attr, use_kwargs_dict

<IPython.core.display.Javascript object>

In [ ]:
# hide
from fastcore.all import *

<IPython.core.display.Javascript object>

## Helpers

In [ ]:
# export
@torch.no_grad()
def maybe_convert_to_onehot(activations: Tensor, labels: Tensor):
    "converts `labels` to onehot Tensors"
    if len(labels.shape) < len(activations.shape):
        onehot = torch.zeros_like(activations)
        onehot.scatter_(1, labels[..., None], 1)
        labels = onehot
    return labels

<IPython.core.display.Javascript object>

In [ ]:
activations = torch.randn(32, 5, requires_grad=True)
labels = torch.empty(32, dtype=torch.long).random_(5)
onehots = maybe_convert_to_onehot(activations, labels)
test_eq(onehots.shape, activations.shape)

<IPython.core.display.Javascript object>

In [ ]:
# export
def log_t(u, t):
    """Compute log_t for `u`."""

    if t == 1.0:
        return torch.log(u)
    else:
        return (u ** (1.0 - t) - 1.0) / (1.0 - t)

<IPython.core.display.Javascript object>

In [ ]:
# export
def exp_t(u, t):
    """Compute exp_t for `u`."""

    if t == 1.0:
        return torch.exp(u)
    else:
        return torch.relu(1.0 + (1.0 - t) * u) ** (1.0 / (1.0 - t))

<IPython.core.display.Javascript object>

In [ ]:
# export
def compute_normalization_fixed_point(activations, t, num_iters=5):
    """Returns the normalization value for each example (t > 1.0).
    Args:
    activations: A multi-dimensional tensor with last dimension `num_classes`.
    t: Temperature 2 (> 1.0 for tail heaviness).
    num_iters: Number of iterations to run the method.
    Return: A tensor of same rank as activation with the last dimension being 1.
    """

    mu = torch.max(activations, dim=-1).values.view(-1, 1)
    normalized_activations_step_0 = activations - mu

    normalized_activations = normalized_activations_step_0
    i = 0
    while i < num_iters:
        i += 1
        logt_partition = torch.sum(exp_t(normalized_activations, t), dim=-1).view(-1, 1)
        normalized_activations = normalized_activations_step_0 * (
            logt_partition ** (1.0 - t)
        )

    logt_partition = torch.sum(exp_t(normalized_activations, t), dim=-1).view(-1, 1)

    return -log_t(1.0 / logt_partition, t) + mu

<IPython.core.display.Javascript object>

In [ ]:
# export
def compute_normalization(activations, t, num_iters=5):
    """Returns the normalization value for each example.
    Args:
    activations: A multi-dimensional tensor with last dimension `num_classes`.
    t: Temperature 2 (< 1.0 for finite support, > 1.0 for tail heaviness).
    num_iters: Number of iterations to run the method.
    Return: A tensor of same rank as activation with the last dimension being 1.
    """

    if t < 1.0:
        # not implemented as these values do not occur in the authors experiments...
        return None
    else:
        return compute_normalization_fixed_point(activations, t, num_iters)

<IPython.core.display.Javascript object>

In [ ]:
# export
def tempered_softmax(activations, t, num_iters=5):
    """Tempered softmax function.
    Args:
    activations: A multi-dimensional tensor with last dimension `num_classes`.
    t: Temperature tensor > 0.0.
    num_iters: Number of iterations to run the method.
    Returns:
    A probabilities tensor.
    """

    if t == 1.0:
        normalization_constants = torch.log(torch.sum(torch.exp(activations), dim=-1))
    else:
        normalization_constants = compute_normalization(activations, t, num_iters)

    return exp_t(activations - normalization_constants, t)

<IPython.core.display.Javascript object>

In [ ]:
# export
class LabelSmoothingCrossEntropy(nn.Module):
    "Cross Entropy Loss with Label Smoothing"

    def __init__(
        self, eps: float = 0.1, reduction: str = "mean", weight: Optional[Tensor] = None
    ):
        super(LabelSmoothingCrossEntropy, self).__init__()
        store_attr("eps, reduction, weight")

    def forward(self, output: Tensor, target: Tensor):
        c = output.size()[1]
        log_preds = F.log_softmax(output, dim=1)
        if self.reduction == "sum":
            loss = -log_preds.sum()
        else:
            loss = -log_preds.sum(dim=1)
            if self.reduction == "mean":
                loss = loss.mean()
        loss = loss * self.eps / c + (1 - self.eps) * F.nll_loss(
            log_preds, target.long(), weight=self.weight, reduction=self.reduction
        )
        return loss

<IPython.core.display.Javascript object>

In [ ]:
lmce = LabelSmoothingCrossEntropy(reduction="mean")
output = torch.randn(32, 5, requires_grad=True)
target = torch.empty(32, dtype=torch.long).random_(5)
loss = lmce(output, target)
test_eq(loss.shape, [])

<IPython.core.display.Javascript object>

In [ ]:
# export
class FocalLoss(nn.CrossEntropyLoss):
    """
    Same as nn.CrossEntropyLoss but with focal paramter, gamma. Focal loss is introduced by Lin et al.
    https://arxiv.org/pdf/1708.02002.pdf.
    """

    @use_kwargs_dict(keep=True, weight=None, ignore_index=-100, reduction="mean")
    def __init__(self, *args, gamma: Union[float, int] = 2, **kwargs):
        super(FocalLoss, self).__init__(**kwargs)
        self.gamma = gamma
        self.reduce = kwargs.pop("reduction") if "reduction" in kwargs else "mean"
        super().__init__(*args, reduction="none", **kwargs)

    def forward(self, output: Tensor, target: Tensor):
        ce_loss = super().forward(output, target)
        pt = torch.exp(-ce_loss)
        fl_loss = (1 - pt) ** self.gamma * ce_loss
        return (
            fl_loss.mean()
            if self.reduce == "mean"
            else fl_loss.sum()
            if self.reduce == "sum"
            else fl_loss
        )

<IPython.core.display.Javascript object>

In [ ]:
fn = FocalLoss(reduction="mean")
output = torch.randn(32, 5, requires_grad=True)
target = torch.empty(32, dtype=torch.long).random_(5)
loss = fn(output, target)
test_eq(loss.shape, [])

<IPython.core.display.Javascript object>

In [ ]:
# export
class SigmoidFocalLoss(nn.Module):
    "Focal Loss with Sigmoid Activation used in RetinaNet for dense detection: https://arxiv.org/abs/1708.02002."

    def __init__(
        self,
        alpha: float = -1,
        gamma: float = 2,
        reduction: str = "mean",
    ):
        super(SigmoidFocalLoss, self).__init__()
        store_attr("alpha, gamma, reduction")

    def forward(self, output: Tensor, target: Tensor):
        target = maybe_convert_to_onehot(output, target)
        loss = sigmoid_focal_loss(
            output, target, self.gamma, self.alpha, self.reduction
        )
        return loss

<IPython.core.display.Javascript object>

Note,  
`alpha` is the Weighting factor in range (0,1) to balance positive vs negative examples. Default = -1 (no weighting).  
`gamma` is Exponent of the modulating factor `(1 - p_t)` to balance easy vs hard examples.

In [ ]:
fn = SigmoidFocalLoss()
output = torch.randn(32, 5, requires_grad=True)
target = torch.empty(32, dtype=torch.long).random_(5)
loss = fn(output, target)
test_eq(loss.shape, [])

<IPython.core.display.Javascript object>

In [ ]:
# export
def bi_tempered_logistic_loss(
    activations, labels, t1, t2, label_smoothing=0.0, num_iters=5, reduction="mean"
):
    """Bi-Tempered Logistic Loss with custom gradient.
    Args:
    activations: A multi-dimensional tensor with last dimension `num_classes`.
    labels: A tensor with shape and dtype as activations.
    t1: Temperature 1 (< 1.0 for boundedness).
    t2: Temperature 2 (> 1.0 for tail heaviness, < 1.0 for finite support).
    label_smoothing: Label smoothing parameter between [0, 1).
    num_iters: Number of iterations to run the method.
    Returns:
    A loss tensor.
    """
    if label_smoothing > 0.0:
        num_classes = labels.shape[-1]
        labels = (
            1 - num_classes / (num_classes - 1) * label_smoothing
        ) * labels + label_smoothing / (num_classes - 1)

    probabilities = tempered_softmax(activations, t2, num_iters)

    temp1 = (log_t(labels + 1e-10, t1) - log_t(probabilities, t1)) * labels
    temp2 = (1 / (2 - t1)) * (
        torch.pow(labels, 2 - t1) - torch.pow(probabilities, 2 - t1)
    )
    loss_values = temp1 - temp2

    if reduction == "none":
        return loss_values
    if reduction == "sum":
        return loss_values.sum()
    if reduction == "mean":
        return loss_values.mean()

<IPython.core.display.Javascript object>

In [ ]:
# export
class BiTemperedLogisticLoss(nn.Module):
    "Implementation of [Robust Bi-Tempered Logistic Loss Based on Bregman Divergences](https://arxiv.org/abs/1906.03361)"

    def __init__(self, eps: float = 0.1, t1=0.6, t2=1.4, num_iters=5, reduction="mean"):
        super(BiTemperedLogisticLoss, self).__init__()
        self.eps = eps
        self.t1, self.t2 = t1, t2
        self.num_iters = num_iters
        self.reduction = reduction

    def forward(self, output, target):
        target = maybe_convert_to_onehot(output, target)
        loss = bi_tempered_logistic_loss(
            output, target, self.t1, self.t2, self.eps, self.num_iters, self.reduction
        )
        return loss

<IPython.core.display.Javascript object>

In [ ]:
fn = BiTemperedLogisticLoss(reduction="mean")
output = torch.randn(32, 5, requires_grad=True)
target = torch.empty(32, dtype=torch.long).random_(5)
loss = fn(output, target)
test_eq(loss.shape, [])

<IPython.core.display.Javascript object>

In [ ]:
# hide
notebook2script()

Converted 00_config.ipynb.
Converted 00a_core.utils.common.ipynb.
Converted 00b_core.utils.data.ipynb.
Converted 00c_core.optim.ipynb.
Converted 00d_core.schedules.ipynb.
Converted 00e_core.layers.ipynb.
Converted 00f_core.losses.py.ipynb.
Converted 01a_classification.data.transforms.ipynb.
Converted 01b_classification.data.datasets.ipynb.
Converted 01c_classification.modelling.backbones.ipynb.
Converted 01d_classification.modelling.classifiers.ipynb.
Converted 01e_classification.modelPL.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>